In [ ]:
import os

import cartopy
import cartopy.crs as ccrs
import contextily as cx
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas
import rasterio
from rasterio.plot import show as rioshow
import xarray as xr

from ghaa.config import load_config
from ghaa.plot.map import get_axes, plot_basemap, plot_basemap_labels, scale_bar

In [ ]:
base_path = load_config()["base_path"]
base_path

In [ ]:
color_codes = {
    "ocean": "#bfc0bf",
    "land": "#e4e4e3",
    "flash flooding moderate scenario": "#a2cfe3",
    "flash flooding high scenario": "#0088b7",
    "landslide": "#d5c68e",
    "drought": "",
    "healthcare": "#9f71a4",
    "exposure": "#d53c17",
    "buildings": "#e3e2de",
    "electricity": "#0f6cb2",
    "water": "#87cefa",
    "transport": "#ffd700"
}

In [ ]:
ax = get_axes()
plot_basemap(ax, os.path.join(base_path, 'data'), plot_regions=True)
plot_basemap_labels(ax, os.path.join(base_path, 'data'), include_regions=True)
scale_bar(ax)

ax

## Raster plot from access CSV

In [ ]:
pop_path = os.path.join(
    base_path, 'incoming', 'population', 'Population Density& Demographics', 
    'ID 23_Population_Density_Demographics - fb AI', 'population_gha_2019-07-01_geotiff', 
    'population_gha_2019-07-01.tif')

In [ ]:
access_path = os.path.join(
    base_path, 'results', 'proximity_results', 'population_gha_2019-07-01_proximity.csv')

In [ ]:
access_tif_path = os.path.join(
    base_path, 'results', 'proximity_results', 'population_gha_2019-07-01_proximity_distance.tif')

In [ ]:
with rasterio.open(pop_path) as ds:
    pass

In [ ]:
Z = np.full(ds.shape, -999.0)

In [ ]:
access = pandas.read_csv(access_path, usecols=['population', 'time_hr', 'access', 'x', 'y'])
access.head()

In [ ]:
for obs in access.itertuples():
    row, col = ds.index(obs.x, obs.y)
    Z[row, col] = obs.time_hr

In [ ]:
with rasterio.open(
        access_tif_path,
        'w',
        driver='GTiff',
        height=Z.shape[0],
        width=Z.shape[1],
        count=1,
        dtype=Z.dtype,
        crs=ds.crs,
        transform=ds.transform,
        nodata=-999.0
    ) as new_dataset:
    new_dataset.write(Z, 1)

In [ ]:
ds = rasterio.open(access_tif_path)

In [ ]:
states = geopandas.read_file(
    os.path.join(
        base_path, 'data', 'admin', 'GHA_admin0.gpkg')
    ).to_crs(4326)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
rioshow((ds, 1), cmap='viridis', ax=ax)
states.plot(ax=ax, facecolor='#00000000', edgecolor='#dddddd')
plt.axis('off')

In [ ]:
left = -0.5
bottom = 5.4
right = 0.1
top = 5.8
from rasterio.windows import from_bounds
with rasterio.open(access_tif_path) as src:
    window = from_bounds(left, bottom, right, top, src.transform)
    rst = src.read(1, window=window)

In [ ]:
import shapely.geometry
import matplotlib.patheffects as path_effects

import matplotlib.cm
import matplotlib.colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
src.crs

In [ ]:
states.to_crs(src.crs)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
cmap = 'viridis'
rioshow(rst, cmap=cmap, ax=ax, transform=src.window_transform(window))


fig = ax.figure
size="3%"
pad=0.09
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size=size, pad=pad)
norm = matplotlib.colors.Normalize(vmin=rst.min(), vmax=rst.max())
mappable = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)
fig.colorbar(mappable, cax=cax)

bbpoly = shapely.geometry.box(left, bottom, right, top)
clipped_states = geopandas.clip(states, bbpoly)
clipped_states.plot(ax=ax, facecolor='#00000000', edgecolor='#dddddd')

ax.axis('off')
ax.text(
    -0.25, 5.6, 'Accra', color='white', size=16,
    path_effects=[
        path_effects.Stroke(linewidth=3, foreground='black'),
        path_effects.Normal()
    ]
)



ax

In [ ]:
# GeoTIFF to plot
tif_path = os.path.join(
    base_path, 'results', 'proximity_results',
    'population_gha_2019-07-01_proximity_distance.tif')

plot_raster(
    tif_path, base_path
)

In [ ]:
# Accra
left = -0.5
bottom = 5.4
right = 0.1
top = 5.8
extent = (left, bottom, right, top)